In [561]:
import pandas, numpy

N_ROWS = 3


# II. Normalizing Addresses:
## Abstract 
Some address are very similar and ought to be grouped together as duplicates. 

By removing punctuation and countries and re-ordering the words in the address string 2112 address (2% of the address database) where found as direct duplicates. 

Furthermore, using fuzzy matching 40,257 addresses (45% of the address database) were found over 75% similar to 11,119 unique addresses from a total of 93,454.

Direct duplicates are cleansed from the 'edges.csv' while the similarities, not being exact duplicates and thus prone to errors, are added to a new 'similarity.edges.csv'.

## 0. Import:

In [562]:
address = pandas.read_csv('data/raw/nodes.address.csv', usecols=['node_id', 'address', 'countries'], index_col='node_id')
address['parent_id'] = 0

address.head( N_ROWS )

,address,countries,parent_id
node_id,,,
14000001,-\t27 ROSEWOOD DRIVE #16-19 SINGAPORE 737920,Singapore,0
14000002,"""Almaly Village"" v.5, Almaty Kazakhstan",Kazakhstan,0
14000003,"""Cantonia"" South Road St Georges Hill Weybridg...",United Kingdom,0


So far, very few duplicates are found:

In [563]:
def count_duplicates(df, col):
    duplicates = df[ df.duplicated([col]) ]

    total = df.shape[0]
    duplicate_count = duplicates.shape[0]
    
    duplicate_percent = int( (duplicate_count * 100) / total )

    print( f'Total entries: {total} \nDuplicates: {duplicate_count} or {duplicate_percent}%' )

count_duplicates(address, 'address')

Total entries: 93454 
Duplicates: 1 or 0%


## 1. Sanitization:
### a. Removing punctuation:
Loading relevant columns, all strings to lowercase, removing punctuation.
Furthermore an empty parent_id column is created.

In [564]:
address = address.applymap( lambda _str: _str.lower() if isinstance( _str,str ) else _str )

def remove_punctuation(_str):
    for punc in [ ';', '#', ',', '|', '\t', '"', '/', '.', '(', ')', '-', '^', '&', '_' ]:
        _str = _str.replace( punc, ' ' )
    return _str
    
address.address = address.address.map( lambda _str: remove_punctuation(_str) if isinstance( _str,str) else _str )

count_duplicates(address, 'address')

Total entries: 93454 
Duplicates: 66 or 0%


### b. Removing countries:

Removing the country from the address strings:

The more edge cases we remove the better results we get from computing similarities.

TODO: Extract and remove cities for better results; { '60 piers road; borrowdale; harare': ('12 hunt road borrowdale, harare', 79), ('12 hunt road; borrrowdale; harare', 77) }

In [565]:
country_list = list( address.countries.dropna().unique() )

### add edge cases ###
st_vincent = ['st. vincent & grenadines', 'st vincent and the grenadines', 'st vincent & the grenadines', 
            'saint-vincent and grenadines', 'saint-vincent', 'st. vincent', 'saint vincent', '& the grenadines']
country_list = st_vincent + country_list
country_list += ['vietnam', 'sengal', 'europe', 'republic of', 'zimbawe', 'zimabwe', 'city']

def remove_country(text, *country_list):
    if type(text) == float:
        return text 
    for country in country_list:
        if country in text:
            text = text.replace(country, '')
    return text.strip()

address.address = address['address'].apply(remove_country, args=country_list)

count_duplicates(address, 'address')

Total entries: 93454 
Duplicates: 643 or 0%


### c. Reordering tokens:

In [566]:
def reorder_tokens(_str):
    as_list = _str.split()
    if len(as_list) > 0:
        return ' '.join(sorted(as_list))
    else: 
        return numpy.NaN
    
address.address = address.address.map( lambda _str: reorder_tokens(_str) if isinstance( _str,str) else _str )

count_duplicates(address, 'address')

Total entries: 93454 
Duplicates: 2297 or 2%


## 2. Deduplication:
Selecting duplicate address/countries pairs with an address string over one word long.

In [567]:
duplicates = address.loc[address.duplicated(['address', 'countries'], keep=False)]

duplicates = duplicates.loc[duplicates.address.str.split().str.len() > 1]

duplicates.sort_values(by='address').head(N_ROWS)

,address,countries,parent_id
node_id,,,
14048910,0 004 1669 235 aires alazan buenos campanillas...,argentina,0
14087441,0 004 1669 235 aires alazan buenos campanillas...,argentina,0
14003887,00 069542 14 143 building cecil gb street,singapore,0


Maping the parent's ids to their duplicates.

In [568]:
def map_parent_id(df):
    # parent_id is the first row's node_id
    parent_id = df.index[0]
    # children are the rest of the rows
    df.iloc[ 1:, df.columns.get_indexer(['parent_id']) ] = parent_id
    return df

def gapply_parallel(df_group, func):
    t1 = time.time()

    df_list = [ group for name, group in df_group ]

    with Pool( cpu_count() ) as pool:
        results = pool.map(func, df_list)

    result_df = pandas.concat( results )

    print( f'* Total in:', time.time()-t1 )
    return result_df

duplicate_group = duplicates.groupby(['address', 'countries'])

matches = gapply_parallel(duplicate_group, map_parent_id)

address.update(matches)
address.parent_id = address.parent_id.astype('Int64')

* Total in: 1.814671516418457


In [569]:
duplicates_mapped = address[ address.parent_id > 0]
print('Duplicates mapped:', duplicates_mapped.shape[0])

duplicates_mapped.head(N_ROWS)

Duplicates mapped: 2112


,address,countries,parent_id
node_id,,,
14000069,1 mapp street,belize,14000068
14000108,193 25 box british complex ground mason o p st...,anguilla,14000094
14000264,02nd cyber ebene ebene floor mews,mauritius,14000263


We save these results directly on the edges previously cleaned up.

In [570]:
edges = pandas.read_csv('data/processing/edges.csv')
duplicate_nodes = duplicates_mapped.index

def duplicate_to_parent(_id):
    if _id in duplicate_nodes:
        return duplicates_mapped.loc[_id, 'parent_id']
    else :
        return _id
    
edges['start_id'] = edges['start_id'].apply(duplicate_to_parent)
edges['end_id'] = edges['end_id'].apply(duplicate_to_parent)

edges.to_csv('data/processing/edges.csv', index=False)

## 3. Fuzzy matching:

#### !! Computationly heavy !!
It is recomended to run this script on anything else than a Thinkpad X270 (Elapsed time: 50min), Amazon AWS being one of the better options. This notebook is thus deliberately limited to groups containing less than 50 strings.

In [571]:
address = address.drop(duplicates_mapped.index).dropna()

### a. The matching algorythm:
### TODO: include a detailed explanation.

In [572]:
from multiprocessing import Pool, cpu_count
from fuzzywuzzy import fuzz, process
import time

THRESHOLD = 75
SELECTOR = 'address'
GROUP = address.groupby('countries')
LIMIT = 0

def calculate_string_similarity(df):
    t1 = time.time()

    choices = set( df[ SELECTOR ].unique() )
    seen = set()
    results = dict()

    choice_count = len(choices)
    for i in range( choice_count-1 ):
        choice = list(choices)[i]
        if len(choice) < 2:
            i += 1
            continue

        seen.add(choice)
        new_choices = choices.difference(seen)
        if len(new_choices) <= 1:
            break

        res = process.extract(choice, new_choices, scorer=fuzz.token_sort_ratio, limit=10000)
        res = [ r[0] for r in res if r[1] > THRESHOLD ]

        if len(res):
            seen.update(res)
            results[choice] = res
        i += 1 

    tt = time.time() - t1
    print( f'Compared {choice_count} in {tt} sec' )
    return results

def gapply_parallel(df_group, func):
    t1 = time.time()

    if LIMIT:
        df_list = [ group for name, group in df_group if group.shape[0] < LIMIT ]
    else:
        df_list = [ group for name, group in df_group ]

    results = {}
    with Pool( cpu_count() ) as pool:
        for res in pool.map(func, df_list):
            results.update( res )

    results = pandas.DataFrame.from_dict(results,  orient='index')
    results.reset_index(inplace=True)
    #results = results.rename( {'index':'parent'} , axis=1)

    print( f'Total in:', time.time()-t1 )
    return results


similar_str = gapply_parallel(GROUP, calculate_string_similarity)
similar_str = similar_str.rename( {'index':'parent'} , axis=1)

Compared 23 in 0.017104387283325195 sec
Compared 17 in 0.0029027462005615234 sec
Compared 10 in 0.0012526512145996094 sec
Compared 34 in 0.03534340858459473 sec
Compared 38 in 0.06016707420349121 sec
Compared 100 in 0.11222243309020996 sec
Compared 17 in 0.01827096939086914 sec
Compared 790 in 3.3133206367492676 sec
Compared 57 in 0.03792572021484375 sec
Compared 19 in 0.005887269973754883 sec
Compared 45 in 0.02748870849609375 sec
Compared 55 in 0.03510165214538574 sec
Compared 222 in 0.5638706684112549 sec
Compared 197 in 0.32608747482299805 sec
Compared 7 in 0.0012884140014648438 sec
Compared 147 in 0.19158315658569336 sec
Compared 34 in 0.020790815353393555 sec
Compared 3 in 0.00063323974609375 sec
Compared 81 in 0.11040520668029785 sec
Compared 657 in 4.988184690475464 sec
Compared 11 in 0.0017247200012207031 sec
Compared 252 in 1.121208906173706 sec
Compared 21 in 0.019019365310668945 sec
Compared 19 in 0.008964300155639648 sec
Compared 71 in 0.07364058494567871 sec
Compared 742 

In [573]:
similar_str.head(N_ROWS)

,parent,0,1,2,3,4,5,6,7,8,...,642,643,644,645,646,647,648,649,650,651
0,31291 al al al box hamra jazeera khaimah o p ras,al al al hamra jazeera khaimah ras,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,al al anónima cien cien de del edificio este j...,al al alimenticia anónima cien cien del edific...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,33b la of ordino pleta principality,33 b la of ordino pleta principality,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### b. String matches to addresss edges:
Map all address strings to their node_id, using all cores available.

In [574]:
def string_to_ID(df):
    def func(x):
        if isinstance(x, str):
            return address.index[ address.address == x ][0]
        return 0
        
    return df.applymap(func)

def apply_parallel(df, func):
    t1 = time.time()

    _cpu_count = cpu_count()
    df_split = numpy.array_split( df, _cpu_count )

    with Pool( _cpu_count ) as pool:
        res = pool.map( func, df_split )
        try:
            df = pandas.concat( res )
        except ValueError:
            # result could be a list of Nones
            pass 
            
    print(f'Total time:', time.time()-t1 )
    return df

similar_nodes = apply_parallel(similar_str, string_to_ID)
similar_nodes.head(N_ROWS)

Total time: 242.22062349319458


,parent,0,1,2,3,4,5,6,7,8,...,642,643,644,645,646,647,648,649,650,651
0,14062504,14026529,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14103880,14103793,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14047924,14047925,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


As parent/child pair:

In [575]:
similar_nodes = ( similar_nodes.astype('Int64').where(similar_nodes.ne(0))
                    .set_index('parent')
                    .stack()
                    .reset_index(level=0, name='child_id') )
                 
similar_nodes.rename( { 'parent' : 'parent_id'}, axis=1, inplace=True)
similar_nodes.reset_index(drop=True, inplace=True)

similar_nodes.head(N_ROWS)

,parent_id,child_id
0,14062504,14026529
1,14103880,14103793
2,14047924,14047925


### c. Resolving problems to better quantify the results:
Quantifying the results. Problems of type "master->parent->child" trees are counted.

In [576]:
def quantify_results():
    original_count = address.shape[0]

    parents = set( similar_nodes.parent_id )
    unique_parents = len( parents )

    children = set( similar_nodes.child_id )
    unique_children = len( children )

    problems = list( parents.intersection( children ))
    problem_count = len( problems )
    problem_percent = int( (problem_count * 100) / unique_parents )

    remaining = original_count - unique_children + problem_count
    reduction = int( 100 - (( remaining * 100 ) / original_count ))

    print( f"""* Reduced {unique_children} to {unique_parents} from a total of {original_count}.
            \n* Problems found: {problem_count} or {problem_percent}% of the parents.
            \n* Remaining: {remaining}. Reduction: -{reduction}%""" )
    return problems

problems = quantify_results()

* Reduced 40257 to 15347 from a total of 90408.
            
* Problems found: 4229 or 27% of the parents.
            
* Remaining: 54380. Reduction: -39%


The next bit of code enables you to trace these problems. 

In [577]:
def trace_problem(prob):
    as_parent = similar_nodes[ similar_nodes.parent_id == prob ]
    as_child = similar_nodes[ similar_nodes.child_id == prob ]

    print(f'As parent: \n{as_parent}\nAs child:\n{as_child}')

    masters = list( as_child.parent_id )
    for i in range( len(masters) ): 
        master = similar_nodes[ similar_nodes.parent_id == masters[i] ]
        i += 1
        print(f'Master {i}:\n{master}')

    print('\nTrace address:')
    print( list( address[ address.index == prob ]['address'] ))
    print('\nMaster address:')
    print( list( address[ address.index == as_child.parent_id.iloc[0]]['address'] ))

trace_problem(problems[6])

As parent: 
     parent_id  child_id
199   14049280  14035572
As child:
     parent_id  child_id
120   14042728  14049280
Master 1:
     parent_id  child_id
120   14042728  14049280

Trace address:
['1210 aires buenos capital federal maipú']

Master address:
['1 10 36 aires buenos capital federal florida oficina piso']


The children's ownership could be transfered to the masters as follow; 

Unfortunately, this operation has to be single threaded as to not modify both the ownership of the masters and parents at the same time.

In [578]:
t1 = time.time()

for prob in problems:

    as_parent = similar_nodes[ similar_nodes.parent_id == prob ]
    as_child = similar_nodes[ similar_nodes.child_id == prob ]
    master_id = list( as_child['parent_id' ])[0]
    
    similar_nodes.loc[similar_nodes.parent_id == prob, 'parent_id'] = master_id   
tt = time.time() - t1

print( f'Total in: {tt}')

problems = quantify_results()

Total in: 21.470188856124878
* Reduced 40257 to 11119 from a total of 90408.
            
* Problems found: 1 or 0% of the parents.
            
* Remaining: 50152. Reduction: -44%


By grouping "master->parent->child" trees, we achive a 45% reduction of the address dataset.

## 4. Saving the results;
The results are saved to "data/processing/similarity.edges.csv".

In [582]:
similar_nodes['type'] = 'similarity'
similar_nodes['text'] = 'similar address'

In [583]:
similar_nodes.to_csv('data/processing/similarity.edges.csv', index=False)